In [ ]:
from dotenv import load_dotenv

load_dotenv()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

In [ ]:
from openai import OpenAI

client = OpenAI()
messages = []
messages.append({"role": "system", "content": "You are a helpful assistant."})

def send_message(message: str):
    messages.append({"role": "user", "content": message})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message

for i in range(1,3):
    answer = send_message(input(f"{i} - 질문을 해주세요:"))
    print(f"{answer.content}\n----------------------------\n")

    messages.append({"role": "assistant", "content": answer.content})


In [ ]:
from langchain_community.chat_models import ChatOpenAI, ChatOllama
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI( temperature=1.0, model="gpt-4o-mini" )
#llm = ChatOllama( temperature=1.0, model="llama3:latest" )

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant DD-BOT."),
    ("ai","Hi. My name is DD-BOT"),
    ("human", "{question}\n그리고 너를 뭐라고 부르면 될까?")
])

prompt = template.format_messages(question="선릉과 정릉에 대해 설명해줘.")
answer = llm.invoke(prompt)
print(f"{answer.content}\n----------------------------\n")

In [ ]:

# 위의 llm.invoke 로 바로 불러도 되지만, 아래처럼 chain 으로 부르면 더 편해짐
chain = template | llm
answer = chain.invoke({"question": "선릉과 정릉에 대해 설명해줘."})
print(f"{answer.content}\n----------------------------\n")


In [ ]:
# 결과를 정리하는 Parser 를 만들어서, 랭체인에 추가할 수 있음.
# DD 에서 task annotation 으로 추가도 가능.

from langchain.schema import BaseOutputParser
from ddtrace.llmobs.decorators import task

class CommaOutputParser(BaseOutputParser):
    @task()
    def parse(self, text: str):
        items = text.split(",")
        return list(map(str.strip, items))
    
template = ChatPromptTemplate.from_messages([
    ("system", "너는 List를 만드는 기계야. 앞으로 답변은 모두 comma로 구분짓고, 최대 {max_items} 개까지만 답변해. 다른 것은 답변하지마."),
    ("human", "{question}")
])

chain = template | llm | CommaOutputParser() 

chain.invoke({
    "max_items":5,
    "question":"조선시대 왕들을 읇어봐"
})

In [ ]:
# 랭체인으로 LLM Call 의 결과도 다시 사용가능

first_template = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant."),
  ("human", "{question}")
])

first_chain = first_template | llm
question = "선릉에 대해 설명해줘."
#result = first_chain.invoke({"question":question})
#result

pirate_prompt = ChatPromptTemplate.from_messages([
    ("system","너는 이제 조선시대 노비야, 이제 아래 들어오는 human 메세지를 조선시대 노비처럼 말해줘."),
    ("human","{human_message}")
])

pirate_chain = pirate_prompt | llm

final_chain = {"human_message":first_chain} | pirate_chain

result = final_chain.invoke({"question":question})
result.content

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS, Chroma

loader = TextLoader("./kings.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder( 
    separator="\n\n",
    chunk_size=1500, 
    chunk_overlap=100
)
docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()

# 문서 임베딩해서 벡터디비로 반환
vector_store = FAISS.from_documents(docs, embedder)


In [ ]:

# 질문 임베딩 후에, 유사 문서 검색
query = "연산군과 관련된 것을 알려줘."
results = vector_store.similarity_search_with_relevance_scores(query)
results


In [ ]:
# 쿼리와 유사한 문서들을 가져오고, 문서들을 토대로 답변하기
from langchain.schema.runnable import RunnablePassthrough

retriever = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 잘 답변하는 어시스턴트야. 이 뒤에 있는 입력된 문맥을 이용해서, 질문에 답변해줘. 답을 모른다면, 그냥 모른다고 답해. 억지롤 답을 만들면 안돼.\n\n{context}"),
    ("human", "{question}")
])

chain = (
    {
        "context":retriever, 
        "question":RunnablePassthrough()
    } 
    | prompt 
    | llm 
)

chain.invoke("어린 나이에 아버지를 잃은 왕을 일려줘.")


In [ ]:
# 쿼리와 유사한 문서들을 가져오고, 문서들을 Re-Ranking 해서 답변하기
from langchain.chains import RetrievalQA
from ddtrace.llmobs.decorators import workflow

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vector_store.as_retriever() 
)

@workflow
def answer_question(query):
    LLMObs.annotate(
        input_data=query,
    )
    result = chain.invoke(query)
    return result['result']

query = "조선 왕들 중 가장 많은 자식을 가진 왕?"
#query = "조선 왕들 중 자식 사랑이 남달랐던 왕에 대해 설명해줘."
answer_question(query)
